In [ ]:
import pandas as pd
import numpy as np

# Read the care area data
care_area_data = pd.read_csv('CareAreas.csv', header=None, names=['ID', 'Xmin', 'Xmax', 'Ymin', 'Ymax'])

# Define the main field and sub field sizes
main_field_size = 100.0
sub_field_size = 2.56

# Function to generate main fields that cover multiple care areas
def generate_main_fields(care_area_data, main_field_size):
    main_fields = []
    care_area_data['MainFieldID'] = np.nan

    x_min = care_area_data['Xmin'].min()
    x_max = care_area_data['Xmax'].max()
    y_min = care_area_data['Ymin'].min()
    y_max = care_area_data['Ymax'].max()

    x_steps = int(np.ceil((x_max - x_min) / main_field_size))
    y_steps = int(np.ceil((y_max - y_min) / main_field_size))

    main_field_id = 0
    for i in range(x_steps):
        for j in range(y_steps):
            x1 = x_min + i * main_field_size
            y1 = y_min + j * main_field_size
            x2 = min(x1 + main_field_size, x_max)
            y2 = min(y1 + main_field_size, y_max)

            intersecting_areas = care_area_data[
                ((care_area_data['Xmin'] < x2) & (care_area_data['Xmax'] > x1)) &
                ((care_area_data['Ymin'] < y2) & (care_area_data['Ymax'] > y1))
            ]

            if not intersecting_areas.empty:
                main_field = {
                    'ID': main_field_id,
                    'Xmin': x1,
                    'Xmax': x2,
                    'Ymin': y1,
                    'Ymax': y2
                }
                main_fields.append(main_field)
                care_area_data.loc[intersecting_areas.index, 'MainFieldID'] = main_field_id
                main_field_id += 1

    return pd.DataFrame(main_fields)

# Function to generate sub fields within the main fields
def generate_sub_fields(main_fields, care_area_data, sub_field_size):
    sub_fields = []
    visited_coordinates = set()

    for main_field in main_fields.itertuples():
        intersecting_areas = care_area_data[care_area_data['MainFieldID'] == main_field.ID]

        for care_area in intersecting_areas.itertuples():
            x_start, y_start = care_area.Xmin, care_area.Ymin
            x_end, y_end = care_area.Xmax, care_area.Ymax
            x_steps = int(np.ceil((x_end - x_start) / sub_field_size))
            y_steps = int(np.ceil((y_end - y_start) / sub_field_size))

            for i in range(x_steps):
                for j in range(y_steps):
                    x1 = x_start + i * sub_field_size
                    y1 = y_start + j * sub_field_size
                    x2 = min(x1 + sub_field_size, x_end)
                    y2 = min(y1 + sub_field_size, y_end)

                    if (x1, y1) not in visited_coordinates:
                        sub_field = {
                            'ID': len(sub_fields),
                            'Xmin': x1,
                            'Xmax': x2,
                            'Ymin': y1,
                            'Ymax': y2,
                            'Main Field ID': main_field.ID
                        }
                        sub_fields.append(sub_field)

                        visited_coordinates.add((x1, y1))
                        visited_coordinates.add((x2, y2))

    return pd.DataFrame(sub_fields)

# Generate main fields and sub fields
main_fields = generate_main_fields(care_area_data, main_field_size)
sub_fields = generate_sub_fields(main_fields, care_area_data, sub_field_size)

# Save the results to CSV files
main_fields.to_csv('MainFields2.csv', index=False)
sub_fields.to_csv('SubFields2.csv', index=False)

print("Main Fields:")
print(main_fields)

print("Sub Fields:")
print(sub_fields)


In [8]:
from google.colab import files
files.download('MainFields2.csv')
files.download('SubFields2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
from google.colab import files
files.download('MainFields2.csv')
files.download('SubFields2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>